<a href="https://colab.research.google.com/github/ianhernandezz/Capstone_Final_Report/blob/main/Capstone_ML_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#connect to Big Query 
from google.colab import auth
auth.authenticate_user()
from google.cloud.bigquery_storage import types


import pandas as pd

In [2]:
#connect to Big Query 
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1
import pandas as pd
%load_ext google.colab.data_table


In [3]:
pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

In [4]:
#connect to Big Query 
!pip install pyarrow==3.0.0

import os
os.environ.setdefault("GCLOUD_PROJECT", "restaurant-scores-345323")

bqclient = bigquery.Client()

# Download query results.
query_string = """
SELECT * FROM `restaurant-scores-345323.restaurant1596.restaurant_data`
"""

dataframe = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        
    )
)
print(dataframe.head())

   business_postal_code_1   inspection_id inspection_date  inspection_score  \
0                   94130  38139_20161214        December                87   
1                   94130  38139_20161214        December                87   
2                   94130  38139_20161214        December                87   
3                   94102    102_20161103        November                85   
4                   94102    102_20161103        November                85   

                               violation_description  risk_category  \
0                      No hot water or running water      High Risk   
1  Wiping cloths not clean or properly stored or ...       Low Risk   
2  Inadequately cleaned or sanitized food contact...  Moderate Risk   
3  Food safety certificate or food handler card n...       Low Risk   
4        Unclean or unsanitary food contact surfaces      High Risk   

   neighborhoods  Current_Police_Districts  population  avg_income  
0             36             

In [5]:
# Initial imports.
import sqlalchemy
import numpy as np
from pathlib import Path
from collections import Counter
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sqlalchemy import create_engine


In [6]:
import imblearn
print(imblearn.__version__)

0.8.1


In [7]:
risk_df1 = pd.DataFrame(dataframe)

In [8]:
risk_df1

,business_postal_code_1,inspection_id,inspection_date,inspection_score,violation_description,risk_category,neighborhoods,Current_Police_Districts,population,avg_income
0,94130,38139_20161214,December,87,No hot water or running water,High Risk,36,1,1453.0,80959.0
1,94130,38139_20161214,December,87,Wiping cloths not clean or properly stored or ...,Low Risk,36,1,1453.0,80959.0
2,94130,38139_20161214,December,87,Inadequately cleaned or sanitized food contact...,Moderate Risk,36,1,1453.0,80959.0
3,94102,102_20161103,November,85,Food safety certificate or food handler card n...,Low Risk,19,6,28991.0,22351.0
4,94102,102_20161103,November,85,Unclean or unsanitary food contact surfaces,High Risk,19,6,28991.0,22351.0
...,...,...,...,...,...,...,...,...,...,...
20239,94134,70685_20171212,December,86,Inadequate and inaccessible handwashing facili...,Moderate Risk,91,2,40134.0,54342.0
20240,94134,70685_20190315,March,88,Moderate risk food holding temperature,Moderate Risk,91,2,40134.0,54342.0
20241,94134,70685_20190315,March,88,Unapproved or unmaintained equipment or utensils,Low Risk,91,2,40134.0,54342.0
20242,94134,70685_20190315,March,88,Unsanitary employee garments hair or nails,Low Risk,91,2,40134.0,54342.0


In [9]:
 # Drop inspection_id
risk_df1 = risk_df1.drop(columns=['inspection_id', 'violation_description'])

In [10]:
risk_df1

,business_postal_code_1,inspection_date,inspection_score,risk_category,neighborhoods,Current_Police_Districts,population,avg_income
0,94130,December,87,High Risk,36,1,1453.0,80959.0
1,94130,December,87,Low Risk,36,1,1453.0,80959.0
2,94130,December,87,Moderate Risk,36,1,1453.0,80959.0
3,94102,November,85,Low Risk,19,6,28991.0,22351.0
4,94102,November,85,High Risk,19,6,28991.0,22351.0
...,...,...,...,...,...,...,...,...
20239,94134,December,86,Moderate Risk,91,2,40134.0,54342.0
20240,94134,March,88,Moderate Risk,91,2,40134.0,54342.0
20241,94134,March,88,Low Risk,91,2,40134.0,54342.0
20242,94134,March,88,Low Risk,91,2,40134.0,54342.0


In [11]:
#change risk_category to binary values. Label 'Moderate Risk' and 'Low Risk' == 0

y_binary = []

for i in risk_df1['risk_category']:
  if i == 'High Risk':
    y_binary.append(1)

  elif i == 'Moderate Risk':
    y_binary.append(0)

  else:
    y_binary.append(0) 

risk_df1['risk_category'] = y_binary

In [12]:
risk_df1['risk_category'].value_counts()

0    17419
1     2825
Name: risk_category, dtype: int64

In [13]:
risk_df1

,business_postal_code_1,inspection_date,inspection_score,risk_category,neighborhoods,Current_Police_Districts,population,avg_income
0,94130,December,87,1,36,1,1453.0,80959.0
1,94130,December,87,0,36,1,1453.0,80959.0
2,94130,December,87,0,36,1,1453.0,80959.0
3,94102,November,85,0,19,6,28991.0,22351.0
4,94102,November,85,1,19,6,28991.0,22351.0
...,...,...,...,...,...,...,...,...
20239,94134,December,86,0,91,2,40134.0,54342.0
20240,94134,March,88,0,91,2,40134.0,54342.0
20241,94134,March,88,0,91,2,40134.0,54342.0
20242,94134,March,88,0,91,2,40134.0,54342.0


In [14]:
# Create our features. 
X = risk_df1.drop(columns='risk_category')

# Create our target
y = risk_df1['risk_category']
y.value_counts()

0    17419
1     2825
Name: risk_category, dtype: int64

In [15]:
X.describe()

,business_postal_code_1,inspection_score,neighborhoods,Current_Police_Districts,population,avg_income
count,20244.000000,20244.000000,20244.000000,20244.000000,20244.000000,20244.000000
mean,94114.061203,85.344892,57.263485,5.399081,37738.252420,51372.542482
std,9.350442,8.110656,35.014539,2.681820,21784.166592,17502.372234
min,94102.000000,46.000000,1.000000,1.000000,374.000000,14609.000000
25%,94108.000000,81.000000,28.000000,3.000000,23016.000000,40990.000000
50%,94111.000000,87.000000,53.000000,6.000000,33115.000000,53795.000000
75%,94121.000000,92.000000,96.000000,8.000000,55492.000000,61362.000000
max,94134.000000,100.000000,117.000000,10.000000,74633.000000,95313.000000


In [16]:
#view X
X

,business_postal_code_1,inspection_date,inspection_score,neighborhoods,Current_Police_Districts,population,avg_income
0,94130,December,87,36,1,1453.0,80959.0
1,94130,December,87,36,1,1453.0,80959.0
2,94130,December,87,36,1,1453.0,80959.0
3,94102,November,85,19,6,28991.0,22351.0
4,94102,November,85,19,6,28991.0,22351.0
...,...,...,...,...,...,...,...
20239,94134,December,86,91,2,40134.0,54342.0
20240,94134,March,88,91,2,40134.0,54342.0
20241,94134,March,88,91,2,40134.0,54342.0
20242,94134,March,88,91,2,40134.0,54342.0


In [17]:
# encode Month as a number
months_num = {
   "January": 1,
   "February": 2,
   "March": 3,
   "April": 4,
   "May": 5,
   "June": 6,
   "July": 7,
   "August": 8,
   "September": 9,
   "October": 10,
   "November": 11,
   "December": 12,
}

In [18]:
X["months_num"] = X["inspection_date"].apply(lambda x: months_num[x])

In [19]:
months_num["June"]

6

In [20]:
X

,business_postal_code_1,inspection_date,inspection_score,neighborhoods,Current_Police_Districts,population,avg_income,months_num
0,94130,December,87,36,1,1453.0,80959.0,12
1,94130,December,87,36,1,1453.0,80959.0,12
2,94130,December,87,36,1,1453.0,80959.0,12
3,94102,November,85,19,6,28991.0,22351.0,11
4,94102,November,85,19,6,28991.0,22351.0,11
...,...,...,...,...,...,...,...,...
20239,94134,December,86,91,2,40134.0,54342.0,12
20240,94134,March,88,91,2,40134.0,54342.0,3
20241,94134,March,88,91,2,40134.0,54342.0,3
20242,94134,March,88,91,2,40134.0,54342.0,3


In [21]:
# drop inspection_date
X = X.drop(columns=['inspection_date'])
X.head()

,business_postal_code_1,inspection_score,neighborhoods,Current_Police_Districts,population,avg_income,months_num
0,94130,87,36,1,1453.0,80959.0,12
1,94130,87,36,1,1453.0,80959.0,12
2,94130,87,36,1,1453.0,80959.0,12
3,94102,85,19,6,28991.0,22351.0,11
4,94102,85,19,6,28991.0,22351.0,11


In [22]:
X.shape

(20244, 7)

In [23]:
# Split data to test and train 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.80)


In [24]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16195, 7)
(4049, 7)
(16195,)
(4049,)


In [25]:
from collections import Counter
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [26]:
Counter(y_resampled)

Counter({0: 12294, 1: 7347})

In [27]:
#Counter(y_resampled)

In [28]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=250) 

In [29]:
# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

In [30]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_resampled)


In [31]:
predictions

array([0, 0, 0, ..., 1, 1, 1])

In [32]:
from sklearn.metrics import confusion_matrix
predictions = rf_model.predict(X_test)
confusion_matrix(y_test, predictions)

array([[2641,  831],
       [ 153,  424]])

In [33]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.7569770313657693

In [34]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])


In [35]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2641,831
Actual 1,153,424


Accuracy Score : 0.7569770313657693
Classification Report
              precision    recall  f1-score   support

           0       0.95      0.76      0.84      3472
           1       0.34      0.73      0.46       577

    accuracy                           0.76      4049
   macro avg       0.64      0.75      0.65      4049
weighted avg       0.86      0.76      0.79      4049



In [36]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

0.7477460186568059

In [37]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      0.76      0.73      0.84      0.75      0.56      3472
          1       0.34      0.73      0.76      0.46      0.75      0.56       577

avg / total       0.86      0.76      0.74      0.79      0.75      0.56      4049



In [38]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.02811623, 0.5753546 , 0.11548599, 0.03943046, 0.02614082,
       0.02379335, 0.19167854])

In [39]:
# sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X_resampled.columns), reverse=True)

[(0.5753545994589024, 'inspection_score'),
 (0.19167854379832447, 'months_num'),
 (0.11548599197622304, 'neighborhoods'),
 (0.03943046374243972, 'Current_Police_Districts'),
 (0.028116232885233097, 'business_postal_code_1'),
 (0.026140822229643625, 'population'),
 (0.023793345909233557, 'avg_income')]